In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install git+https://github.com/NVIDIA/NeMo.git

In [ ]:
!pip install git+https://github.com/NVIDIA/NeMo-text-processing

In [ ]:
!python3 -m pip install python-dev-tools --user --upgrade

In [ ]:
!python3 -m venv hydra-env
!source hydra-env/bin/activate
!cd hydra-env/
!pip install --upgrade pip setuptools wheel   # Didn't need this on Fedora 32 but also works with it
!pip install hydra-core --upgrade

In [ ]:
!pip install lhotse einops pytorch_lightning jiwer pyannote.audio webdataset datasets wandb dill

In [ ]:
# Load spectrogram generator
import numpy as np
from nemo.collections.tts.models import FastPitchModel
spec_generator = FastPitchModel.from_pretrained("tts_en_fastpitch_multispeaker")

# Load Vocoder
from nemo.collections.tts.models import HifiGanModel
model = HifiGanModel.from_pretrained(model_name="tts_en_hifitts_hifigan_ft_fastpitch")

spec_generator.eval()
model.eval()

In [ ]:
!ls -l /content/drive/MyDrive/Capstone/TestFiles/test | wc -l
!ls -l /content/drive/MyDrive/Capstone/TestFiles/train | wc -l

In [ ]:
!pip install memory_profiler

In [ ]:
# Generate audio
import soundfile as sf
import os
import random
import gc
import cProfile
from memory_profiler import profile
import sys
import io

@profile
def generate_file_set(filename, f, out_dir ):
  max_speakers = 20
  file1 = open(f, 'r')
  text = file1.read().strip()
  file1.close()
  # Generate a random number between 1 and max_speakers as speaker id
  speaker_id = random.randint(1, max_speakers)
  parsed = spec_generator.parse(text)
  spectrogram = spec_generator.generate_spectrogram(tokens=parsed, speaker=speaker_id)
  audio = model.convert_spectrogram_to_audio(spec=spectrogram)
  # Save the audio to disk in a file called speech.wav
  audio = model.convert_spectrogram_to_audio(spec=spectrogram)
  audio = audio.to('cpu').detach().numpy()
  filename = os.path.splitext(filename)[0]
  audio_file = "speech-" + filename + "-" + str(speaker_id) + ".wav"
  rand_num = random.randint(0,4)
  if ((speaker_id > 18) or (rand_num == 2) ):
    output_directory = out_dir + 'test/'
  else:
    output_directory = out_dir + 'train/'
  sf.write(output_directory + audio_file, np.ravel(audio), 44100)
  text_file = "speech-" + filename + "-" + str(speaker_id) + ".txt"

  f = open(output_directory + text_file, "w")
  f.write(text)
  f.close()
  del spectrogram
  del audio
  gc.collect()


def generate_audio():

  output_directory = '/content/drive/MyDrive/Capstone/TestFiles/'

  # assign directory
  for index in range(243, 250):
    directory = '/content/drive/MyDrive/Capstone/VCTK/txt/p' + str(index)
    # Check if the directory exists
    if os.path.exists(directory) and os.path.isdir(directory):
      # iterate over files in
      # that directory
      for filename in os.listdir(directory):
          f_value = os.path.join(directory, filename)
          # checking if it is a file
          if os.path.isfile(f_value):
            #cProfile.runctx('generate_file_set(filename, f_value, output_directory)', globals(), locals())
            # Redirect stdout to capture memory profiler output
            stdout_backup = sys.stdout
            sys.stdout = io.StringIO()

            # Run the function
            generate_file_set(filename, f_value, output_directory)

            # Get the captured memory profiler output
            memory_profiler_output = sys.stdout.getvalue()

            # Restore the original stdout
            sys.stdout = stdout_backup

            # Print the memory profiler output
            print(memory_profiler_output)

generate_audio()
